# Parsing the property sales data stored in “data.dat”
The real estate markets, like those in Sydney and Melbourne, present an interesting opportunity for data analysts to analyze and predict where property prices are moving towards.  Prediction of property prices is becoming increasingly important and beneficial. Property prices are a good indicator of both the overall market condition and the economic health of a country. Considering the data provided, we are wrangling a large set of property sales records stored in an unknown format and with unknown data quality issues.

### Examining and loading the data into a Pandas DataFrame

In [ ]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize


with open("../input/data.dat") as json_file:
    json_data = json.load(json_file)
    houses=pd.DataFrame(json_data)

In [ ]:
#The loaded JSON data has been saved in a Python dictionary. 
#Using json_normalize, flattening the "data" dictionary into a table and saving it in a DataFrame "df".
df=json_normalize(json_data['houses'])
print(df.head(5))

In [ ]:
df['date'].value_counts()

From the above value counts of "date", we found that there are two dates with different date format when compared to other date formats.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#plotting bar graph to audit "date"
df['date'].value_counts().plot(kind="line",figsize=(15,5))
plt.show()

# there is a different date formate for "23052014T000000" replacing it with "20140523T000000"
df['date'].replace(['23052014T000000'],['20140523T000000'],inplace=True) 

# there is a date value with 20140631T000000, where as June month does not contain 31st and hence considering it has Irregularities. changing date 20140631T000000 to 20140701T000000  
df['date'].replace(['20140631T000000'],['20140701T000000'],inplace=True)


##### Splitting the "address" values to => "street","city", "statezip", "country"

In [ ]:
#'address' is split according to "," and stored into 'address_list'
df['address_list'] = df['address'].str.split(', ')
#from 'address_list', col[0] is asigned to street,col[1] is asigned to city
#col[2] is asigned to statezip and col[3] is asigned to country
df['street'] = df['address_list'].apply(lambda col: col[0])
df['city']=df['address_list'].apply(lambda col: col[1])
df['statezip']=df['address_list'].apply(lambda col: col[2])
df['country']=df['address_list'].apply(lambda col: col[3])
df.head(3)

In [ ]:
#Dropping 'address' column and also dummy list created as 'address_list'
df.drop('address',axis=1, inplace=True)
df.drop('address_list', axis=1, inplace = True)
df.head(3)

In [ ]:
#"room" attribute values are extracted into 'bathrooms' and 'bedrooms',and "room" attribute is dropped once its values are extracted
df['bathrooms'] = df.rooms.str.extract('Number of bathrooms: (\d.\d+)', expand = True)
df['bedrooms'] = df.rooms.str.extract('Number of bedrooms: (\d+)', expand = True)
df.drop('rooms', axis=1, inplace = True)
df.head(3)


In [ ]:
#splitting the values of 'area.sqft_living/sqft_lot' according to "="
df['area.sqft_living/sqft_lot_list'] = df['area.sqft_living/sqft_lot'].str.split('=')
# col[1] has the values of sqft_living and sqft_lot, hence storing it in 'area.sqft_living/sqft_lot_list_list1'
df['area.sqft_living/sqft_lot_list_list1'] = df['area.sqft_living/sqft_lot_list'].apply(lambda col: col[1])
df['area.sqft_living/sqft_lot_list_list2'] = df['area.sqft_living/sqft_lot_list_list1'].str.split('\ ')
df['sqft_living']=df['area.sqft_living/sqft_lot_list_list2'].apply(lambda col: col[0])
df['sqft_lot']=df['area.sqft_living/sqft_lot_list_list2'].apply(lambda col: col[1])
#dropping all dummy list used to store while splitting values of 'area.sqft_living/sqft_lot'
df.drop('area.sqft_living/sqft_lot_list_list1',axis=1, inplace=True)
df.drop('area.sqft_living/sqft_lot_list_list2', axis=1, inplace = True)
df.drop('area.sqft_living/sqft_lot_list', axis=1, inplace = True)
df.drop('area.sqft_living/sqft_lot', axis=1, inplace = True)

#renaming the columns from "area.sqft_above" to "sqft_above" and "area.sqft_basement" to "sqft_basement"
df.rename(index=str, columns={"area.sqft_above": "sqft_above", "area.sqft_basement": "sqft_basement"},inplace=True)
df.head(3)

In [ ]:
#right stripping the value of 'sqft_living'
df['sqft_living'] = df['sqft_living'].map(lambda x: x.rstrip('\\'))
df.head(2)

In [ ]:
#Trying find the Irregularities in sqft_living by applying, sqft_basement + sqft_above = sqft_living
df['sqft_temp'] =  df[['sqft_basement', 'sqft_above']].sum(axis=1)

#comparing the temp values with 'sqft_living'
df[df['sqft_temp'] != df['sqft_living']].index
#print(df.iloc[[4338]])

#df.ix[4338, 'sqft_living'] = df.ix[4338,['sqft_basement', 'sqft_above']].sum()

#print(df.iloc[[4338]])
#print(df.iloc[[4339]])

#df.ix[4339, 'sqft_living'] = df.ix[4339,['sqft_basement', 'sqft_above']].sum()

#print(df.iloc[[4339]])

In [ ]:
#changing the datetime format.
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%dT%H:%m:%s')

In [ ]:
#Changing the data types
df[['bedrooms', 'bathrooms']] = df[['bedrooms', 'bathrooms']].astype(float)
df[['sqft_lot','sqft_living']] = df[['sqft_lot','sqft_living']].astype(np.int64)
df['price'] = df['price'].apply(np.int64)
df.info()

In [ ]:
#rearranging the dataframe according to the requirement
df = df[['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 
         'condition', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'street', 'city', 'statezip', 
         'country']]

In [ ]:
df.head()

# Auditing and cleansing the loaded data
In this task, we are inspecting and auditind the data to identify the data problems, and then fix the problems. Different generic and major data problems could be found in the data might include:

* Lexical errors, e.g., typos and spelling mistakes
* Irregularities, e.g., abnormal data values and data formats
* Violations of the Integrity constraint.
* Outliers
* Duplications
* Missing values
* Inconsistency, e.g., inhomogeneity in values and types in representing the same data

In [ ]:
 # 'O' for Objects
#df.describe(include=['O'])
df.describe(include="all")

###  Investigating Lexical errors for all the columes in dataframe

In [ ]:
df['city'].value_counts()

From the above data, when we observe the unique values, Some of the city names were Lexical errors.

In [ ]:
#City
# there is a Lexical error in city "sammamish" which is replaced with the average value of city named "Sammamish"
df['city'].replace(['sammamish'],['Sammamish'],inplace=True) 

# there is a Lexical error in city "Samamish" which is replaced with the average value of city named "Sammamish"
df['city'].replace(['Samamish'],['Sammamish'],inplace=True) 

# there is a Lexical error in city "Seaattle" which is replaced with the average value of city named "Seattle"
df['city'].replace(['Seaattle'],['Seattle'],inplace=True) 

# there is a Lexical error in city "Seatle" which is replaced with the average value of city named "Seattle"
df['city'].replace(['Seatle'],['Seattle'],inplace=True) 
 
# there is a Lexical error in city "seattle" which is replaced with the average value of city named "Seattle"
df['city'].replace(['seattle'],['Seattle'],inplace=True)

# there is a Lexical error in city "Issaguah" which is replaced with the average value of city named "Issaquah"
df['city'].replace(['Issaguah'],['Issaquah'],inplace=True)

# there is a Lexical error in city "Woodenville" which is replaced with the average value of city named "Woodinville"
df['city'].replace(['Woodenville'],['Woodinville'],inplace=True)
 
# there is a Lexical error in city "redmond" which is replaced with the average value of city named "Redmond"
df['city'].replace(['redmond'],['Redmond'],inplace=True)

# there is a Lexical error in city "Redmund" which is replaced with the average value of city named "Redmond"
df['city'].replace(['Redmund'],['Redmond'],inplace=True)

# there is a Lexical error in city "Redmund" which is replaced with the average value of city named "Redmond"
df['city'].replace(['Redmonde'],['Redmond'],inplace=True)

# there is a Lexical error in city "auburn" which is replaced with the average value of city named "Auburn"
df['city'].replace(['auburn'],['Auburn'],inplace=True)

# there is a Lexical error in city "Auburnt" which is replaced with the average value of city named "Auburn"
df['city'].replace(['Auburnt'],['Auburn'],inplace=True)

# there is a Lexical error in city "Sureline" which is replaced with the average value of city named "Shoreline "
df['city'].replace(['Sureline'],['Shoreline'],inplace=True)

# there is a Lexical error in city "Bellvue" which is replaced with the average value of city named "Bellevue "
df['city'].replace(['Bellvue'],['Bellevue'],inplace=True)

# there is a Lexical error in city "Belleview" which is replaced with the average value of city named "Bellevue "
df['city'].replace(['Belleview'],['Bellevue'],inplace=True)

# there is a Lexical error in city "Snogualmie" which is replaced with the average value of city named "Snoqualmie"
df['city'].replace(['Snogualmie'],['Snoqualmie'],inplace=True)

# there is a Lexical error in city "Coronation" which is replaced with the average value of city named "Carnation"
df['city'].replace(['Coronation'],['Carnation'],inplace=True)

# there is a Lexical error in city "Kirklund" which is replaced with the average value of city named "Kirkland"
df['city'].replace(['Kirklund'],['Kirkland'],inplace=True)

#The above changes can aslo be done as show in below code,
#df.city.replace({"sammamish":"Sammamish", "Samamish": "Sammamish", "Seaattle":"Seattle", "Seatle":"Seattle",
#"seattle":"Seattle", "Issaguah":"Issaquah"}, inplace=True) 
df.city.value_counts()

In [ ]:
df['bathrooms'].value_counts()

From the above data, when we observe the unique values, Some of the "bathrooms" values were Irregular.

In [ ]:
#Bathroom
# there is a abnormal data value in "bathrooms", "1.70" which is replaced with the average value"1.75"
df['bathrooms'].replace([1.70],[1.75],inplace=True) 

# there is a lexical error in "bathrooms", "1.05" which is replaced with the value "1.50"
df['bathrooms'].replace([1.05],[1.50],inplace=True) 

# there is a abnormal data value in "bathrooms", "2.55" which is replaced with the nearest value of bathrooms named "2.50"
df['bathrooms'].replace([2.55],[2.50],inplace=True) 

# there is a abnormal data value in "bathrooms", "2.30" which is replaced with the average value of bathrooms named "2.25"
df['bathrooms'].replace([2.30],[2.25],inplace=True) 

# there is a lexical error in "bathrooms", "2.57" which is replaced with the average value of bathrooms named "2.75"
df['bathrooms'].replace([2.57],[2.75],inplace=True) 

df['bathrooms'].value_counts()

### Investigating Duplicates

In [ ]:
df[df.duplicated(keep=False)]

From the above data, we found that there is only one row which is repeating twice.

In [ ]:
#dropping the row which are duplictaes
df.drop_duplicates(keep="first", inplace=True)

### Investigating the missing values 

'nan' doesn't occur in counts() 

*  checking how many 'NaN' values are there
*  checking how many 0
*  checking how many < 1

In [ ]:
df.info()

There are 4601 x 18 records and there are many missing values in "yr_renovated"

In [ ]:
df.isnull().sum()

From the above data, we see that there are 4371 null values in "yr_renovated"

In [ ]:
# unique values of 'yr_renovated'
df['yr_renovated'].unique()

In [ ]:
#creating dummy dataframe to use it for predicting the null values for yr_renovated by using mean
df_impute= df.copy()
df_impute.head(5)


##### There are two ways that we are trying to replace the null values of "yr_renovated"
* Replacing all NaN with zero 
* Replacing NaN with mean

In [ ]:
#converting the yr_renovated column, from float64 to int64 and replacing all NaN to '0'
df_impute['yr_renovated'] = np.nan_to_num(df_impute['yr_renovated']).astype(np.int64)

##### Predicting the yr_renovated when changed from NaN to zero, with the full data

In [ ]:
#For linear regression we use sklearn (built in python library) and import linear regression from it.
from sklearn.linear_model import LinearRegression
#Initializing Linear Regression to a variable reg
reg = LinearRegression()
#we know that 'yr_renovated' are to be predicted , hence we set labels (output) as 'yr_renovated' column
labels = df_impute['yr_renovated']
#Converting dates to 1’s and 0’s so that it doesn’t influence our data much
#We use 0 for houses which are new that is built after 2014.
conv_dates = [1 if values == 2014 else 0 for values in df_impute.date]
df_impute['date'] = conv_dates
train1 = df_impute.drop(['city','street','country','statezip','price'],axis=1)

In [ ]:
#We again import another dependency to split our data into train and test
from sklearn.cross_validation import train_test_split

In [ ]:
#train data is set to 90% and 10% of the data to be my test data , and randomized the splitting of data by using random_state.
X_train, X_test, y_train, y_test = train_test_split(train1,labels,test_size = 0.10, random_state=2)

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
reg.score(X_test,y_test)

##### Predicting the yr_renovated when NaN is changed with mean values with the full data

In [ ]:
df["yr_renovated"].fillna(df.groupby(["yr_built","condition"])["yr_renovated"].transform("mean"), inplace=True)
df.yr_renovated.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df['yr_renovated'].fillna(0, inplace=True)

In [ ]:
df_impute = df.copy()

In [ ]:
#we know that 'yr_renovated' are to be predicted , hence we set labels (output) as 'yr_renovated' column
labels = df_impute['yr_renovated']
#Converting dates to 1’s and 0’s so that it doesn’t influence our data much
#We use 0 for houses which are new that is built after 2014.
conv_dates = [1 if values == 2014 else 0 for values in df_impute.date]
df_impute['date'] = conv_dates
train1 = df_impute.drop(['city','street','country','statezip','price'],axis=1)

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
reg.score(X_test,y_test)

##### Predicting the yr_renovated without any changes with the full data

In [ ]:
df_imptd1 = df.copy()

In [ ]:
#we know that 'yr_renovated' are to be predicted , hence we set labels (output) as 'yr_renovated' column
labels = df_imptd1['yr_renovated']
#Converting dates to 1’s and 0’s so that it doesn’t influence our data much
#We use 0 for houses which are new that is built after 2014.
conv_dates = [1 if values == 2014 else 0 for values in df_imptd1.date]
df_imptd1['date'] = conv_dates
train1 = df_imptd1.drop(['city','street','country','statezip','price'],axis=1)

In [ ]:
#train data is set to 90% and 10% of the data to be my test data , and randomized the splitting of data by using random_state.
X_train, X_test, y_train, y_test = train_test_split(train1,labels,test_size = 0.10, random_state=2)

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
reg.score(X_test,y_test)

In [ ]:
df['yr_renovated'] = df_impute['yr_renovated'].astype(np.int64)

###### checking how many 0 in price

In [ ]:
df.price.value_counts()

From the above observation, we see that there are 248 count of "0.0" in price


##### checking how many values are < 1

In [ ]:
#checking how many values are < 1
df_impute[df_impute['price'] < 1] 

In [ ]:
df[["price","bedrooms","bathrooms","sqft_living","sqft_lot","sqft_above","yr_built","sqft_living","sqft_lot"]].describe()

We can see the home prices vary from $0 to $2.659000e+07 with living space from 370.000000sqft to 13540sqft. Lots of variety!

###### We are trying to replace the zero values of "price"
Replacing zero with mean

In [ ]:
#replacing all the 0.0 to NaN
df['price'] = df['price'].replace(0.0, np.nan)

In [ ]:
#replacing all NaN to mean values of price 
df["price"].fillna(df_impute.groupby(["bedrooms","bathrooms","city","statezip"])["price"].transform("mean"), inplace=True)
df.price.describe()

In [ ]:
df.isnull().sum()

In [ ]:
#dropping all NaN values from price
df.dropna(subset=['price'],axis=0,inplace=True)

In [ ]:
df_final = df.copy()

### Using boxpot to detect outliers

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

df.boxplot(figsize=(20,10))

The first thing to notice is that 'price' has many outliers.
However, plotting all data together might not be right because of the different ranges of attributes. Therefore, we look at one attribute at a time instead.

In [ ]:
bp = df.boxplot(column='price',figsize=(10,15))

In [ ]:
# We can see a bunch of price above 0.5, then something around 2.5, the outliers are:
df[df['price'] > 2.0] 

price of 26590000.0 and 12899000.0 looks very high compared to other, that's a weird value too and price of 7800.0 and 84350.0 
looks very low compared to other values.
* Price also depends on the other conditions of the house
* Lets check the price according to "price","bedrooms","bathrooms","sqft_living","sqft_lot","sqft_above","yr_built","sqft_living","sqft_lot"

###### Let's investigate outliers by each attribute 

In [ ]:
# plotting baoxplot to check outliers price vs bedrooms
bp = df.boxplot(column='price', by = 'bedrooms',figsize=(15,10))

From the above graph, we can summarize that for a standard three bedroom house, the number of outliers imply that the owners paid a premium price for their respective property, which would have either been at a waterfront or the properties' per sqft_living value would have been higher than standard rates. Figuratively, the sandard rates are below 0.5. Under close observation, above the value of 1.0, there are few properties which may have higher sqft_living value, the values above 2.5 might be of those properties which may be near water front 

To verify the above properties whose sqft_living or their location is at a waterfront, we further plot refined graph on waterfront, sqft_living, bedrooms, bathrooms, view, condition. 

In [ ]:
df[df['price'] > 2.0] 

In [ ]:
#creating a dummy dataframe to check on price vs date of property sold.
df_dummy = df
#df_dummy['date'] = pd.to_datetime(df_dummy['date'])

In [ ]:
# plot price vs date

#df_dummy['Year'] = df_dummy['date'].dt.year
#df_dummy.boxplot(column='price', by='Year', figsize=(10,10))

#df.set_index(['date',df.date.dt.year])['price'].unstack().boxplot()

We can see a bunch of price above 0.5, then something around 1.0 and then something around 2.5

In [ ]:
df[(df['price'] > 0.5) & (df['price'] > 1.0) & (df['price'] > 2.5)].describe()

price of minimum 7.800000e+03 with sqft_living 370.000000, zero bathroom and zero bedroom with condition as 1 which is really low comparing to other property's conditions, 2.659000e+07 with sqft_living 13540.000000 looks very high compared to other, that's a weird value too and maximum price of 2.659000e+07 and 7.800000e+03 looks very low compared to other values.
Price also depends on the other conditions of the house

In [ ]:
# sqft_lot
df.boxplot(column='sqft_lot', figsize=(10,10))

We can see a bunch of sqft_lot above 400000, then something around 600000, look at the outliers:

In [ ]:
df[(df['sqft_lot'] > 400000) & (df['sqft_lot'] > 600000)] 

We can see a bunch of sqft_lot above 1074218, then something around 600000, look at the outliers:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import seaborn as sns
from matplotlib import rcParams

%matplotlib inline 
%pylab inline 

Lets look at "continuous" features. Intuitively that will be sqft_living but could possibly be 'bathrooms','bedrooms','sqft_living','sqft_lot','sqft_above','waterfront'. Lets take a look at these with some plots using seaborn.

In [ ]:
sns.pairplot(data=df, x_vars=['bathrooms','bedrooms','sqft_living','sqft_lot','sqft_above','waterfront'], y_vars=["price"])

We can see that "lot" size is not well correlated to price but the data for living space is reasonable. Visually the best feature to use looks like sqft_living as we expected.

## Multivariate linear regression

It’s important to look at the shape of the data – and to double check if the data is reasonable. Corrupted data is not uncommon,so will run two checks
* first, use df.describe() to look at all the variables in our analysis. 
* Second, plot histograms of the variables that the analysis is targeting using plt.pyplot.hist().

In [ ]:
df.describe()

Quick takeaways: We are working with a dataset that contains 4600 observations, mean price is approximately $5.648432e+05,median price is approximately $4.710000e+05, and the average house’s area is 2139.346957 ft2

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

When we produce a linear regression summary with OLS with only two variables this will be the formula that we use:

Reg = ols(‘Dependent variable ~ independent variable(s), dataframe).fit()

print(Reg.summary())

When we look at housing prices and square footage for houses, we print out the following summary report:

In [ ]:
m = ols('price ~ sqft_living',df).fit()
print (m.summary())

When we print the summary of the OLS regression, all relevant information can be easily found, including R-squared, t-statistics, standard error, and the coefficients of correlation. Looking at the output, it’s clear that there is an extremely significant relationship between square footage and housing prices since there is an extremely high t-value of 144.920, and a P>|t| of 0%–which essentially means that this relationship has a near-zero chance of being due to statistical variation or chance.

This relationship also has a decent magnitude – for every additional 100 square-feet a house 

In [ ]:
m = ols('price ~ sqft_living + bedrooms + view + condition',df).fit()
print (m.summary())

In our multivariate regression output above, we learn that by using additional independent variables, such as the number of bedrooms, we can provide a model that fits the data better, as the R-squared for this regression has increased to 0.222. This means that we went from being able to explain about 0.202(20.2%) of the variation in the model to 0.222(22.2%) with the addition of a few more independent variables. 

Having the regression summary output is important for checking the accuracy of the regression model and data to be used for estimation and prediction – but visualizing the regression is an important step to take to communicate the results of the regression in a more digestible format.

This section will rely entirely on Seaborn (sns) , which has an incredibly simple and intuitive function for graphing regression lines with scatterplots. I chose to create a jointplot for square footage and price that shows the regression line as well as distribution plots for each variable.

In [ ]:
sns.jointplot('sqft_living','price', data=df, size=10, alpha=.5)

The increase of price with sqft_living space is pretty clear and the "Pearson r value" is 0.45 indicating a reasonable correlation. However, the data distributions show a big concentration of values in the lower left of the plot. That makes sense, most houses are between 300 and 3000 sqft and a few hundred thousand dollars. We can eliminate the very expensive and very large houses and take another look at the data.

If we set the size (xlim) from 500 to 3500sqft and the price (ylim) from 100,000 to $1,000,000 the data still shows the trend but it looks very scattered.

In [ ]:
sns.jointplot('sqft_living','price', data=df, xlim=(500,3500), ylim=(100000,1000000), size=10, alpha=.5)

Something worth considering is that different neighborhoods can vary greatly in average house price. Some nice neighborhoods are very expensive and some other (also nice!) neighborhoods can be quite affordable. It might be good to look at average house price by zipcode since we have that in our dataset.

In [ ]:
df["statezip"].nunique()


It looks like there are 77 different statezip code in the given data. Lets see how many house sales there were in each.

In [ ]:
df['statezip'].value_counts()

Trying to find the average house sale price in each zipcode ...

In [ ]:
df.groupby('statezip')['price'].mean()

The zipcode that look the most interesting to me is WA 98103. WA 98103 has the most house sale values, 148, with an average sale price of $5.603248e+05. 


In [ ]:
zip_WA_98103 = df['statezip'] == "WA 98103"  # True if zip is 98103

Using the "selectors" above we can look at plots of price vs sqft_living in that zipcode.

In [ ]:
sns.jointplot('sqft_living','price', data=df[zip_WA_98103], size=10, alpha=.5)

The 98103 zipcode has a distribution that looks similar to the complete dataset.

In [ ]:
df.describe()

# Working with Regression Model

Looking at the bedroom columns , the dataset has a house where the house has 9 bedrooms , seems to be a massive house and would be interesting to know more about it as we progress.

Maximum square feet is 13,540 where as the minimum is 370. we can see that the data is distributed.
Similarly , we can infer so many things by just looking at the describe function.

Now , we are going to see some visualization and also going to see how and what can we infer from visualization.

Let’s see which is most common bedroom number. Let’s look at this problem from a builder’s perspective, sometimes it’s important for a builder to see which is the highest selling house type which enables the builder to make house based on that. For example, in India , for a good locality a builder opts to make houses which are more than 3 bedrooms which attracts the higher middle class and upper class section of the society.
Let’s see how this pans out for this data!


In [ ]:
import seaborn as sns
import mpl_toolkits

#
df['bedrooms'].value_counts().plot(kind='bar')
plt.title('Number of Bedrooms')
plt.xlabel('Bedrooms')
plt.ylabel('Count')
sns.despine


As we can see from the visualization 3 bedroom houses are most commonly sold followed by 4 bedroom. So how is it useful ? For a builder having this data , He can make a new building with more 3 and 4 bedroom’s to attract more buyers.
So now we know that 3 and 4 bedroom’s are highest selling. But at which locality ?
#### How common factors are affecting the price of the houses ?
We saw the common locations and now we’re going to see few common factors affecting the prices of the house and if so ? then by how much ?
Let us start with , If price is getting affecting by living area of the house or not ?

In [ ]:
#PRIcE Vs SQFT_LIVING
plt.scatter(df.price,df.sqft_living)
plt.title("Price Vs Square feet")
plt.xlabel('Square feet')
plt.ylabel('Price')
plt.show()

From the above figure we can see that more the living area , more the price though data is concentrated towards a particular price zone , but from the figure we can see that the data points seem to be in linear direction.We can also see some irregularities that the house with the highest square feet was sold for very less , maybe there is another factor or probably the data must be wrong. 

In [ ]:
#PRICE Vs BEDROOMS
plt.scatter(df.bedrooms,df.price)
plt.title("Bedroom and Price")
plt.xlabel("Bedrooms")
plt.ylabel("Price")
plt.show()
sns.despine()

We can see more factors affecting the price

In [ ]:
#Total sqft including basement vs price and waterfront vs price
plt.scatter((df['sqft_living']+df['sqft_basement']),df['price'])
plt.title("sqft_living and sqft_basement vs Price")
plt.xlabel("sqft_living and sqft_basement")
plt.ylabel("Price")
plt.show()

In [ ]:
plt.scatter(df.waterfront,df.price)
plt.title("Waterfront Vs Price (0 = No Waterfront )")
plt.xlabel("waterfront")
plt.ylabel("Price")
plt.show()

##### Floors vs Price and condition vs Price

In [ ]:
plt.scatter(df.floors,df.price)
plt.title("floors Vs Price")
plt.xlabel("floors")
plt.ylabel("Price")
plt.show()

In [ ]:
plt.scatter(df.condition,df.price)
plt.title("condition Vs Price")
plt.xlabel("condition")
plt.ylabel("Price")
plt.show()

As we can see from all the above representation that many factors are affecting the prices of the house , like square feet which increases the price of the house and even location influencing the prices of the house.
Creating a model to which would predict the price of the house based upon the other factors such as square feet , water front etc .

In [ ]:
#For linear regression we use sklearn (built in python library) and import linear regression from it.
from sklearn.linear_model import LinearRegression
#Initializing Linear Regression to a variable reg
reg = LinearRegression()
#we know that 'yr_renovated' are to be predicted , hence we set labels (output) as 'yr_renovated' column
labels = df['price']
#Converting dates to 1’s and 0’s so that it doesn’t influence our data much
#We use 0 for houses which are new that is built after 2014.
conv_dates = [1 if values == 2014 else 0 for values in df.date]
df['date'] = conv_dates
train1 = df.drop(['city','street','country','statezip','price'],axis=1)


In [ ]:
#We again import another dependency to split our data into train and test
from sklearn.cross_validation import train_test_split

In [ ]:
#train data is set to 90% and 10% of the data to be my test data , and randomized the splitting of data by using random_state.
X_train, X_test, y_train, y_test = train_test_split(train1,labels,test_size = 0.10, random_state=2)

In [ ]:
map(pd.np.shape,[X_train, X_test, y_train, y_test])

We have train data , test data and labels for both. Fitting our train and test data into linear regression model.

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
reg.score(X_test,y_test)


After fitting our data to the model we can check the score of our data ie , prediction. in this case the prediction is 58%

In [ ]:
df_final.info()

In [ ]:
df_final.head()

In [ ]:
filename = 'output.csv'
df_final.to_csv(filename, encoding='utf-8', index=False)

##### References:
* https://stackoverflow.com/
* https://www.coursera.org/learn/ml-regression/lecture/G12Qp/a-case-study-in-predicting-house-prices
* https://medium.com/towards-data-science/create-a-model-to-predict-house-prices-using-python-d34fe8fad88f
* https://www.datacamp.com/courses/cleaning-data-in-python
* http://www.developintelligence.com/blog/2017/08/data-cleaning-pandas-python/
* http://uwescience.github.io/DSSG2015-predicting-permanent-housing/images/DSSG2015-PPH-final-presentation.pdf
